# freqency+tfidf extraction

In [ ]:
!pip install transformers -q
!pip install sentencepiece -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sklearn


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Find the top n frequent words in findings and use them as input to the model

In [ ]:
#load the train file
chexbert_df = pd.read_csv("/content/drive/My Drive/Final project - Zebra/Final data/chexbert&pert reports - train.csv")
chexbert_df = chexbert_df[pd.notna(chexbert_df['report_findings'])]
chexbert_df.reset_index(drop=True, inplace=True)
chexbert_df.head()

,path,chexpert medical tags,report_impression,report_findings,start_report,chexbert medical tags
0,files/p10/p10000032/s50414267.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,"FINDINGS: There is no focal consolidation, ...",FINAL REPORT ...,No Finding positive
1,files/p10/p10000032/s53189527.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary abnorm...,"FINDINGS: The cardiac, mediastinal and hila...",FINAL REPORT ...,No Finding positive
2,files/p10/p10000032/s53911762.txt,No Finding positive,IMPRESSION: No acute intrathoracic process.,FINDINGS: Single frontal view of the chest ...,FINAL REPORT ...,No Finding positive
3,files/p10/p10000032/s56699142.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,FINDINGS: The lungs are clear of focal cons...,FINAL REPORT ...,No Finding positive
4,files/p10/p10000764/s57375967.txt,"Consolidation positive ,Pneumonia uncertain",IMPRESSION: Focal consolidation at the left...,FINDINGS: PA and lateral views of the chest...,FINAL REPORT ...,"Consolidation positive ,Pneumonia uncertain"


In [ ]:
#load the test file
examples_df = pd.read_csv("/content/drive/My Drive/Final project - Zebra/Final data/chexbert&pert reports - test.csv")
examples_df = examples_df[pd.notna(examples_df['report_findings'])]
examples_df.reset_index(drop=True, inplace=True)
examples_df.head()

,path,chexbert medical tags,chexpert medical tags,report_impression,report_findings,start_report
0,files/p10/p10699400/s54816399.txt,No Finding positive,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,FINDINGS: The lungs are clear without consol...,FINAL REPORT ...
1,files/p10/p10699460/s54836969.txt,No Finding positive,No Finding positive,IMPRESSION: No acute intrathoracic abnormality.,FINDINGS: PA and lateral views of the chest ...,FINAL REPORT ...
2,files/p11/p11955295/s51529848.txt,Cardiomegaly positive,Cardiomegaly positive,IMPRESSION: Heart size is borderline enlarg...,"FINDINGS: There is no CHF, consolidation, p...",FINAL REPORT ...
3,files/p13/p13050559/s52657655.txt,"Lung Opacity positive ,Atelectasis positive ,P...","Atelectasis positive ,Lung Opacity positive ,P...",IMPRESSION: 1. Stable radiographic appeara...,FINDINGS: No significant interval change. S...,FINAL REPORT ...
4,files/p13/p13783064/s58234209.txt,"Pneumothorax negative ,Pleural Effusion positive","Pleural Effusion positive ,Pneumothorax negative",IMPRESSION: Left pleural effusion has decrea...,FINDINGS: Single supine frontal view of the ...,FINAL REPORT ...


In [ ]:
#concat both files and extract only findings
all_findings = pd.concat([chexbert_df,examples_df])
all_findings.reset_index(drop=True, inplace=True)
all_findings.head()

,path,chexpert medical tags,report_impression,report_findings,start_report,chexbert medical tags
0,files/p10/p10000032/s50414267.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,"FINDINGS: There is no focal consolidation, ...",FINAL REPORT ...,No Finding positive
1,files/p10/p10000032/s53189527.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary abnorm...,"FINDINGS: The cardiac, mediastinal and hila...",FINAL REPORT ...,No Finding positive
2,files/p10/p10000032/s53911762.txt,No Finding positive,IMPRESSION: No acute intrathoracic process.,FINDINGS: Single frontal view of the chest ...,FINAL REPORT ...,No Finding positive
3,files/p10/p10000032/s56699142.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,FINDINGS: The lungs are clear of focal cons...,FINAL REPORT ...,No Finding positive
4,files/p10/p10000764/s57375967.txt,"Consolidation positive ,Pneumonia uncertain",IMPRESSION: Focal consolidation at the left...,FINDINGS: PA and lateral views of the chest...,FINAL REPORT ...,"Consolidation positive ,Pneumonia uncertain"


In [ ]:
#extract the top n words in a df column
def frequent_words(n, df, column):
    
    #count words in the given column
    count_freq ={}
    for index, report in df.iterrows():
        findings_str = str(report[column]) 
        for word in re.findall(r'\w+', findings_str):
            if word not in count_freq.keys():
                count_freq.update({word:1}) 
            else:
                count_freq[word]+=1
     
    #sort from most frequent to the less
    count_freq = dict(sorted(count_freq.items(), key=lambda item: item[1], reverse = True))   
    #print(count_freq)
    
    #extract only the first n words
    frequent_words = []
    for index, key in enumerate(count_freq):
        if index<n:
            frequent_words.append(key)
        else:
            break
            
    important_frequent_findings_words = ["atelectasis", "pneumonia", "pleural", "effusion", "lung", "opacity", "cardiomediastinum", "edema", "consolidation", "cardiomegaly", "pneumothorax", "tube", "lungs", "opacities", "congestion"]
    for word in important_frequent_findings_words:
      if word not in frequent_words:
        frequent_words.append(word)
    if "___" in frequent_words:
      frequent_words.remove("___")

    return frequent_words

frequent_findings_words_25 = frequent_words(6, all_findings, 'report_findings')
frequent_findings_words_50 = frequent_words(43, all_findings, 'report_findings')
frequent_findings_words_75 = frequent_words(200, all_findings, 'report_findings')


print("the most frequent words in findings section are {}".format(frequent_findings_words_75))


the most frequent words in findings section are ['is', 'the', 'are', 'of', 'and', 'The', 'FINDINGS', 'There', 'pleural', 'in', 'right', 'effusion', 'pneumothorax', 'No', 'or', 'no', 'normal', 'left', 'with', 'to', 'seen', 'lung', 'chest', 'pulmonary', 'consolidation', 'silhouette', 'clear', 'a', 'focal', 'unchanged', 'contours', 'size', 'lungs', 'atelectasis', 'mediastinal', 'lateral', 'edema', 'hilar', 'tube', 'at', 'on', 'prior', 'cardiac', 'cardiomediastinal', 'lower', 'has', 'noted', 'stable', 'within', 'unremarkable', 'heart', 'opacity', 'upper', 'there', 'volumes', 'A', 'acute', 'mild', 'osseous', 'be', 'views', 'pneumonia', 'effusions', 'evidence', 'radiograph', 'opacities', 'study', 'compared', 'limits', 'structures', 'small', 'lobe', 'likely', 'low', 'without', 'from', 'vascular', 'for', 'tip', 'enlarged', 'again', 'Heart', 'bilateral', 'been', 'not', 'base', 'which', 'sided', 'present', 'previous', 'position', 'well', 'identified', 'PA', 'Cardiomediastinal', 'thoracic', 'cong

In [ ]:
def input_only_freq(findings_df, frequent_findings_words):
  freq_findings = ""
  flag_found = False
  count_total_words = 0
  count_words_in_input = 0

  df_output = pd.DataFrame({'freq' : []})


  for i in range (len(findings_df)):
      current_findings = str(findings_df.loc[i,'report_findings']) #extract current findings
      current_findings = current_findings.split(".") #split to sentences


      #iterate through all sentences in findings
      for sen in current_findings:
        words_in_sen = sen.split()
        for word in words_in_sen:
          count_total_words +=1  
          if (word in frequent_findings_words):
            count_words_in_input +=1  
            freq_findings = freq_findings+str(word)+" "
            flag_found = True
        if flag_found == True:
          freq_findings = freq_findings+str(", ")
        flag_found = False
      
      #print(current_findings)
      #print(freq_findings)
      df_output.loc[i, 'freq'] = freq_findings
      freq_findings = ""

  print("input contain only {}% from the findings report".format(count_words_in_input/count_total_words*100))
  return df_output

#Contain on avg 25% of the findings section in each report which are most freq + medical labels
top_25 = input_only_freq(all_findings, frequent_findings_words_25)
top_25_to_examples = top_25[-15:]
top_25_to_examples.reset_index(drop=True, inplace=True)

#Contain on avg 50% of the findings section in each report which are most freq + medical labels
top_50 = input_only_freq(all_findings, frequent_findings_words_50)
top_50_to_examples = top_50[-15:]
top_50_to_examples.reset_index(drop=True, inplace=True)

#Contain on avg 75% of the findings section in each report which are most freq + medical labels
top_75 = input_only_freq(all_findings, frequent_findings_words_75)
top_75_to_examples = top_75[-15:]
top_75_to_examples.reset_index(drop=True, inplace=True)

chexbert_df['freq_25%'] = top_25[:-15]
examples_df['freq_25%'] = top_25_to_examples

chexbert_df['freq_50%'] = top_50[:-15]
examples_df['freq_50%'] = top_50_to_examples

chexbert_df['freq_75%'] = top_75[:-15]
examples_df['freq_75%'] = top_75_to_examples

chexbert_df.head()            

input contain only 27.705083704427054% from the findings report
input contain only 49.56014201185933% from the findings report
input contain only 74.28367399820543% from the findings report


,path,chexpert medical tags,report_impression,report_findings,start_report,chexbert medical tags,freq_25%,freq_50%,freq_75%
0,files/p10/p10000032/s50414267.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,"FINDINGS: There is no focal consolidation, ...",FINAL REPORT ...,No Finding positive,"is pleural effusion pneumothorax , opacities ,...",There is no focal pleural effusion or pneumoth...,There is no focal pleural effusion or pneumoth...
1,files/p10/p10000032/s53189527.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary abnorm...,"FINDINGS: The cardiac, mediastinal and hila...",FINAL REPORT ...,No Finding positive,"The and are , is , are , pleural effusion pneu...",The mediastinal and hilar contours are normal ...,The mediastinal and hilar contours are normal ...
2,files/p10/p10000032/s53911762.txt,No Finding positive,IMPRESSION: No acute intrathoracic process.,FINDINGS: Single frontal view of the chest ...,FINAL REPORT ...,No Finding positive,"of the , is pneumothorax , The is , are the an...","of the chest , There is no focal or pneumothor...","view of the chest provided , There is no focal..."
3,files/p10/p10000032/s56699142.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,FINDINGS: The lungs are clear of focal cons...,FINAL REPORT ...,No Finding positive,The lungs are of pleural effusion pneumothorax...,The lungs are clear of focal pleural effusion ...,The lungs are clear of focal pleural effusion ...
4,files/p10/p10000764/s57375967.txt,"Consolidation positive ,Pneumonia uncertain",IMPRESSION: Focal consolidation at the left...,FINDINGS: PA and lateral views of the chest...,FINAL REPORT ...,"Consolidation positive ,Pneumonia uncertain","and of the , The lungs are , is consolidation ...","and lateral of the chest , The lungs are , The...","PA and lateral views of the chest provided , T..."


In [ ]:
from google.colab import drive
from google.colab import files

chexbert_df.to_csv('findings_freq.csv', index = False)
files.download("findings_freq.csv")

examples_df.to_csv('examples_freq.csv', index = False)
files.download("examples_freq.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Find the top n tf-idf words in findings and use them as input to the model - top n word in each findings section

In [ ]:
#load the train file
chexbert_df = pd.read_csv("/content/drive/My Drive/Final project - Zebra/Final data/chexbert&pert reports - train.csv")
chexbert_df = chexbert_df[pd.notna(chexbert_df['report_findings'])]
chexbert_df.reset_index(drop=True, inplace=True)
print(len(chexbert_df))
chexbert_df.head()

149796


,path,chexpert medical tags,report_impression,report_findings,start_report,chexbert medical tags
0,files/p10/p10000032/s50414267.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,"FINDINGS: There is no focal consolidation, ...",FINAL REPORT ...,No Finding positive
1,files/p10/p10000032/s53189527.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary abnorm...,"FINDINGS: The cardiac, mediastinal and hila...",FINAL REPORT ...,No Finding positive
2,files/p10/p10000032/s53911762.txt,No Finding positive,IMPRESSION: No acute intrathoracic process.,FINDINGS: Single frontal view of the chest ...,FINAL REPORT ...,No Finding positive
3,files/p10/p10000032/s56699142.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,FINDINGS: The lungs are clear of focal cons...,FINAL REPORT ...,No Finding positive
4,files/p10/p10000764/s57375967.txt,"Consolidation positive ,Pneumonia uncertain",IMPRESSION: Focal consolidation at the left...,FINDINGS: PA and lateral views of the chest...,FINAL REPORT ...,"Consolidation positive ,Pneumonia uncertain"


In [ ]:
#load the test file
examples_df = pd.read_csv("/content/drive/My Drive/Final project - Zebra/Final data/chexbert&pert reports - test.csv")
examples_df = examples_df[pd.notna(examples_df['report_findings'])]
examples_df.reset_index(drop=True, inplace=True)
print(len(examples_df))
examples_df.head()


15


,path,chexbert medical tags,chexpert medical tags,report_impression,report_findings,start_report
0,files/p10/p10699400/s54816399.txt,No Finding positive,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,FINDINGS: The lungs are clear without consol...,FINAL REPORT ...
1,files/p10/p10699460/s54836969.txt,No Finding positive,No Finding positive,IMPRESSION: No acute intrathoracic abnormality.,FINDINGS: PA and lateral views of the chest ...,FINAL REPORT ...
2,files/p11/p11955295/s51529848.txt,Cardiomegaly positive,Cardiomegaly positive,IMPRESSION: Heart size is borderline enlarg...,"FINDINGS: There is no CHF, consolidation, p...",FINAL REPORT ...
3,files/p13/p13050559/s52657655.txt,"Lung Opacity positive ,Atelectasis positive ,P...","Atelectasis positive ,Lung Opacity positive ,P...",IMPRESSION: 1. Stable radiographic appeara...,FINDINGS: No significant interval change. S...,FINAL REPORT ...
4,files/p13/p13783064/s58234209.txt,"Pneumothorax negative ,Pleural Effusion positive","Pleural Effusion positive ,Pneumothorax negative",IMPRESSION: Left pleural effusion has decrea...,FINDINGS: Single supine frontal view of the ...,FINAL REPORT ...


In [ ]:
#concat both files and extract only findings
all_findings = pd.concat([chexbert_df['report_findings'],examples_df['report_findings']])
all_findings.reset_index(drop=True, inplace=True)


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer

def tf_idf_words(factor, input_df):

  tfidf_findings = ""
  flag_found = False

  df_output = pd.DataFrame({'tfidf' : []})

  labels_words = pd.Series(["atelectasis", "pneumonia", "pleural", "effusion", "lung", "opacity",\
                                       "cardiomediastinum", "edema", "consolidation", "cardiomegaly", "pneumothorax", "tube", "lungs", "opacities", "congestion"])

  #build tf-idf score to each word in each findings
  tfIdfVectorizer=TfidfVectorizer()
  tfIdf = tfIdfVectorizer.fit_transform(input_df)

  count_total_words = 0 #count all the word across all findings
  count_words_in_input = 0 #count only the words in findings that will be extracted by top tf-idf n words

  #iterate through all findings
  for i in range(len(input_df)):

    #create a dataframe of words and scores from highest to lowest and select the top n words with the highest score
    tf_idf_df = pd.DataFrame(tfIdf[i].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    tf_idf_df = tf_idf_df.sort_values('TF-IDF', ascending=False)
    current_findings = str(input_df[i])
    count_total_words += len(current_findings.split())

    n = int(len(current_findings.split())/factor)
    tf_idf_df = tf_idf_df[:n]
    tfidf_values = tf_idf_df.index.values
    tfidf_values_pd = pd.Series(tfidf_values)

    #add the labels 
    important_words = pd.concat([tfidf_values_pd,  labels_words], axis=0) 

    current_findings = current_findings.split(".") #split to sentences
    #iterate through all sentences in findings
    for sen in current_findings:
      words_in_sen = sen.split()
      for word in words_in_sen:
        if (word in important_words.tolist()):
          count_words_in_input +=1  
          tfidf_findings = tfidf_findings+str(word)+" "
          flag_found = True
      if flag_found == True:
        tfidf_findings = tfidf_findings+str(", ")
      flag_found = False
    
    #print(current_findings)
    #print(tfidf_findings)
    #print(important_words.tolist())
    #print("-----------------------------------------------------------")
    df_output.loc[i,'tfidf'] = tfidf_findings 
    tfidf_findings = ""


  print("input contain only {}% from the findings report".format(count_words_in_input/count_total_words*100))
  return df_output

In [ ]:
#Contain on avg 25% of the findings section in each report with most tdidf score + medical labels
top_25 = tf_idf_words(9, all_findings)
top_25_to_examples = top_25[-15:]
top_25_to_examples.reset_index(drop=True, inplace=True)

#Contain on avg 50% of the findings section in each report with most tdidf score + medical labels
top_50 = tf_idf_words(2.9, all_findings)
top_50_to_examples = top_50[-15:]
top_50_to_examples.reset_index(drop=True, inplace=True)

#Contain on avg 75% of the findings section in each report with most tdidf score + medical labels
top_75 = tf_idf_words(1.6, all_findings)
top_75_to_examples = top_75[-15:]
top_75_to_examples.reset_index(drop=True, inplace=True)

chexbert_df['tfidf_25%'] = top_25[:-15]
examples_df['tfidf_25%'] = top_25_to_examples

chexbert_df['tfidf_50%'] = top_50[:-15]
examples_df['tfidf_50%'] = top_50_to_examples

chexbert_df['tfidf_75%'] = top_75[:-15]
examples_df['tfidf_75%'] = top_75_to_examples



input contain only 22.72278364737279% from the findings report
input contain only 48.12111180559256% from the findings report
input contain only 73.06109102227654% from the findings report


In [ ]:
print(chexbert_df['report_findings'][0])
print(chexbert_df['tfidf_25%'][0])

print(examples_df['report_findings'][0])
print(examples_df['tfidf_25%'][0])

 FINDINGS:   There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted. 
pleural effusion pneumothorax , opacities nipple shadows , breast , sixth seventh , 
 FINDINGS:  The lungs are clear without consolidation or edema.  There is no pleural effusion or pneumothorax.  The cardiomediastinal silhouette is normal. 
lungs without consolidation or edema , pleural effusion or pneumothorax , 


In [ ]:
from google.colab import drive
from google.colab import files

chexbert_df.to_csv('findings_tfidf.csv', index = False)
files.download("findings_tfidf.csv")

examples_df.to_csv('examples_tfidf.csv', index = False)
files.download("examples_tfidf.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>